In [2]:
import sys

sys.path.append('..')

from deeptables.models import deeptable,deepnets
import logging
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.ERROR)
import warnings

warnings.filterwarnings("ignore")

In [3]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

config = deeptable.ModelConfig(nets=deepnets.DCN,auto_discrete=True)

dt = deeptable.DeepTable(config=config)
oof_proba, eval_proba_mean, test_proba_mean = dt.fit_cross_validation(
    dftrain, 
    y_train, 
    X_test=dfeval, 
    num_folds=5, 
    epochs=50)

Start cross validation
2 class detected, {0, 1}, so inferred as a [binary classification] task
Preparing features cost:0.010730981826782227
Imputation cost:0.04131364822387695
Categorical encoding cost:0.03501391410827637
Discretization cost:0.03295612335205078
fit_transform cost:0.12431693077087402
transform X_test
transform_X cost:0.035108089447021484
Iterators:KFold(n_splits=5, random_state=9527, shuffle=True)
Injected a callback [EarlyStopping]. monitor:val_AUC, patience:1, mode:max

Fold:1



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (9)', 'input_continuous_all: (4)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [3, 4, 9, 5, 3, 5, 4, 4, 6]
output_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4]
dropout: 0.3
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
---------------------------------------------------------
concat_embed_dense: shape: (None, 40)
---------------------------------------------------------
nets: ['dcn_nets']
---------------------------------------------------------
dcn-widecross: input_shape (None, 40), output_shape (None, 40)
dcn-dnn2: input_shape (None, 40), output_shape (None, 64)
dcn: input_shape (None, 40), output_shape (None, 104)
---------------------------

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.7min finished


In [5]:
dt.leaderboard()

TypeError: 'DataFrame' object is not callable

In [ ]:
# oof score
roc_auc_score(y_train, oof_proba)

In [ ]:
# test score
roc_auc_score(y_eval,test_proba_mean)